In [ ]:
import momia2 as mo
from skimage import measure, feature, filters, transform, morphology
import numpy as np
import pandas as pd
import read_roi as rr
import matplotlib.pyplot as plt
import glob,os
import pickle as pk

In [83]:
from scipy import ndimage as ndi
from skimage.filters import gabor_kernel
def prepare_gabor_kernels(n_theta=4,sigmas=[2.8],frequencies=[0.23]):
    kernels = []
    for theta in range(n_theta):
        theta = (theta / n_theta) * np.pi
        for sigma in sigmas:
            for frequency in frequencies:
                kernel = np.real(gabor_kernel(frequency, theta=theta,
                                              sigma_x=sigma, sigma_y=sigma))
                kernels.append(kernel)
    return kernels


def min_max(data):
    return (data-data.min())/(data.max()-data.min())

def power(image, kernel):
    # Normalize images for better comparison.
    image = (image - image.mean()) / image.std()
    power_image = np.sqrt(ndi.convolve(image, np.real(kernel), mode='wrap')**2 +
                          ndi.convolve(image, np.imag(kernel), mode='wrap')**2)
    convolved = filters.gaussian(power_image/np.median(power_image),sigma=0.8)
    return convolved

## Convert training set pixels to tabular features 

In [90]:
# train RFC model
training_set_folder = '/Users/jz-rolling/Desktop/Size_calculation/20220118/New_trainingset_20220120/'
gabor_kernels = prepare_gabor_kernels()
all_training_data = []
for seed_file in sorted(glob.glob(training_set_folder+'*_seed.tif')):
    header = seed_file.split('/')[-1].split('_')[0]
    target_file = training_set_folder+'{}.tif'.format(header)
    seed = (plt.imread(seed_file)>0)*1
    target = plt.imread(target_file).astype(int)
    target = mo.utils.dual_bandpass(target,
                                    pixel_microns=1,
                                    min_structure_scale=2,
                                    max_structure_scale=500)
    dilated = (morphology.binary_dilation(seed,morphology.disk(1))>0)*1
    edge = (dilated-seed)
    bg = (seed==0)*1
    x1,y1 = np.where(seed>0)
    x2,y2 = np.where(edge>0)
    x3,y3 = np.where(bg>0)
    x = np.hstack([x1,x2,x3])
    y = np.hstack([y1,x2,y3])
    labeled=np.zeros(target.shape)
    labeled[x2,y2]=2
    labeled[x1,y1]=1
    img_features = mo.metrics.image_feature.multiscale_image_feature(target,sigmas=(1,2.5,5),
                                                                     shapeindex=True,
                                                                     rog=True)[1]
    pixel_stats = mo.metrics.image_feature.local_stat(img_features,x,y)
    for i,g in enumerate(gabor_kernels):
        filtered = power(target, g)
        pixel_stats['Gabor_{}'.format(i)] = filtered[x,y]
    pixel_stats['annotation'] = labeled[x,y]
    all_training_data.append(pixel_stats)
all_training_data = pd.concat(all_training_data)

## Train new colony patch classifier using momia2

In [91]:
model = mo.classify.GenericClassifier(criterion='entropy',
                                      class_weight='balanced',
                                      n_estimators=100, 
                                      max_features=22,
                                      n_jobs=-1)
model.features = all_training_data.columns[:-1]
model.train(all_training_data,test_size=0.4)
mo.classify.classifier.save_model(model,'/Users/jz-rolling/Desktop/Size_calculation/20220118/New_trainingset_20220120/model_20220120.pk')

Classification score using RandomForest model is 0.9246694048979339
